In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [17]:
code = """select A.DRVR_USER_ID, A.MARKET_NM,
B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df["ACTV_DT"] = pd.to_datetime(df["ACTV_DT"])
df.head()

,DRVR_USER_ID,MARKET_NM,ACTV_DUR_MIN_QTY,ACTV_DT
0,r0bertorobertus@yandex.com,Walmart Tulsa,239.470000000,2021-07-03
1,r0bertorobertus@yandex.com,Walmart Tulsa,146.930000000,2021-07-03
2,r0bertorobertus@yandex.com,Walmart Tulsa,183.350000000,2021-07-03
3,r0bertorobertus@yandex.com,Walmart Tulsa,70.520000000,2021-07-03
4,jalebittle@gmail.com,Walmart Tulsa,0.100000000,2021-07-03


In [18]:
df.drop((df.loc[df["DRVR_USER_ID"].str.contains("walmart|faisal",case=False)]).index, inplace = True)

In [19]:
df

,DRVR_USER_ID,MARKET_NM,ACTV_DUR_MIN_QTY,ACTV_DT
0,r0bertorobertus@yandex.com,Walmart Tulsa,239.470000000,2021-07-03
1,r0bertorobertus@yandex.com,Walmart Tulsa,146.930000000,2021-07-03
2,r0bertorobertus@yandex.com,Walmart Tulsa,183.350000000,2021-07-03
3,r0bertorobertus@yandex.com,Walmart Tulsa,70.520000000,2021-07-03
4,jalebittle@gmail.com,Walmart Tulsa,0.100000000,2021-07-03
...,...,...,...,...
1303680,tufa.lolo@yahoo.com,Walmart South Minneapolis,72.730000000,2022-06-28
1303681,tufa.lolo@yahoo.com,Walmart South Minneapolis,85.320000000,2022-06-28
1303682,tufa.lolo@yahoo.com,Walmart South Minneapolis,5.020000000,2022-06-28
1303683,tufa.lolo@yahoo.com,Walmart South Minneapolis,210.000000000,2022-06-28


In [20]:
df = df[df["ACTV_DUR_MIN_QTY"] > 0 ]
df = df.drop_duplicates()

In [21]:
df = df[df['ACTV_DT'] >= '2022-05-01']

In [22]:
df = df.reset_index(drop=True)

In [23]:
df

,DRVR_USER_ID,MARKET_NM,ACTV_DUR_MIN_QTY,ACTV_DT
0,mswhitmore@icloud.com,Walmart Tulsa,125.780000000,2022-05-08
1,shikiwoods@gmail.com,Walmart Tulsa,122.930000000,2022-05-08
2,shikiwoods@gmail.com,Walmart Tulsa,1.980000000,2022-05-08
3,shikiwoods@gmail.com,Walmart Tulsa,0.330000000,2022-05-08
4,shikiwoods@gmail.com,Walmart Tulsa,242.670000000,2022-05-08
...,...,...,...,...
363015,sktriemert@hotmail.com,Walmart South Minneapolis,81.000000000,2022-06-28
363016,tufa.lolo@yahoo.com,Walmart South Minneapolis,72.730000000,2022-06-28
363017,tufa.lolo@yahoo.com,Walmart South Minneapolis,85.320000000,2022-06-28
363018,tufa.lolo@yahoo.com,Walmart South Minneapolis,5.020000000,2022-06-28


In [25]:
df.groupby(['MARKET_NM','ACTV_DT','DRVR_USER_ID'],as_index=False).sum().to_csv('Desktop/latest_6markets.csv')